In [ ]:
#用于1车道1交通灯和 pytorch测试
#作者lukeliuli@163.com
# -*- coding: utf-8 -*-
"""
@author: lukeliuli@163.com
"""
#%cd /root/myPrjs/SUMONBDT
import torch
import torchvision
import torchvision.transforms as transforms

import torchvision.models as models
import torchvision.datasets as datasets
import torch.nn as nn
import torch.nn.functional as F

import matplotlib.pyplot as plt
import numpy as np
import torch.optim as optim

import time
from collection
import numpy as np
import os
import sys

import random
import csv
import datetime


def sortTakeSecond(elem):
    return elem[1] 



os.environ['SUMO_HOME'] = '/home/pi/myPrjs/sumo'
if 'SUMO_HOME' in os.environ:
     tools = os.path.join(os.environ['SUMO_HOME'], 'tools')
     sys.path.append(tools)
else:
     sys.exit("please declare environment variable 'SUMO_HOME'")
    


import libsumo

path = os.getcwd()
path2 = path+"\sumoCfgs\my1Lane1TlsVeh3-server.sumocfg"#window
path2 = path+"/sumoCfgs1/my1Lane1TlsVeh3-server.sumocfg"#linux
sumoBinary = "sumo"
libsumo.start([sumoBinary, "-c", path2])


#定义模拟过程中动态可调整参数
params ={}
params["GreenTime"] = 15
params["redTime"] = 25
params["ObjVehPosInXY"] = [-150,0]
params["MaxAcc"] = 1
params["DrvResTime"] = 0.5
params["addedVeh"] = 2+1
params["accel"]=2.8    
params["tau"]=1  
    
numSim = 1000
nextTLSID = "2"
veh = "00003"
counterNoLowSpeed = 0
dataAllSim = []
for i in range(numSim):
    print("###################################################################start sim %d" %i)
    str1 = "./trainData/redTLS_"+str(i)+"_sim.csv"
    csv_file = open(str1,"w+")    
    writer = csv.writer(csv_file)   



    libsumo.start([sumoBinary, "-c", path2])
    stepNum = 0
    requireStop = 0
    redLowSpeed = 0
    data = []
    ##修改出发位置，随机参数
    X =  params["ObjVehPosInXY"][0]+min(random.random()*150,80)#注意坐标的原点
    Y =  params["ObjVehPosInXY"][1]
    libsumo.vehicle.moveToXY(veh,"e1to2", 0, X, Y, -1073741824.0, 1)
    
    ##修改加速，反应时间
    vehOthers = "EBUS2"
    accel = params["accel"]
    accel = max(accel*random.random(),1)
    libsumo.vehicletype.setAccel(vehOthers,accel)
    
    tau = params["tau"]
    tau = max(tau*random.random(),0.3)
    libsumo.vehicletype.setTau(vehOthers,tau)
    
    ####修改绿灯时间，随机参数
    """
    greenTime =  params["GreenTime"]#注意坐标的原点
    greenTime = max(random.random()*greenTime,0.1)
    allProgramLogics = libsumo.trafficlight.getAllProgramLogics(nextTLSID)
    lgc1 = allProgramLogics[0]
    yellowDurTime = lgc1.phases[0].duration
    greenDurTime = lgc1.phases[1].duration
    yellowDurTime = lgc1.phases[2].duration
    redDurTime = lgc1.phases[3].duration
    
    lgc1.phases[1].duration = greenTime
    libsumo.trafficlight.setProgramLogic(nextTLSID,lgc1) 
    phaseDur = libsumo.trafficlight.getPhaseDuration(nextTLSID)
    """
    
    ####修改逻辑和红灯时间，随机参数
    redTime =  params["redTime"]#注意坐标的原点
    redTime = max(random.random()*redTime,0.1)
    allProgramLogics = libsumo.trafficlight.getAllProgramLogics(nextTLSID)
    lgc1 = allProgramLogics[0]
    yellowDurTime = lgc1.phases[0].duration
    
    lgc1.phases[1].state = 'r'
    redDurTime = lgc1.phases[1].duration
    
    yellowDurTime = lgc1.phases[2].duration
    
    lgc1.phases[3].state = 'G'
    greenDurTime = lgc1.phases[3].duration
    
    lgc1.phases[1].duration = redTime
    libsumo.trafficlight.setProgramLogic(nextTLSID,lgc1) 
    phaseDur = libsumo.trafficlight.getPhaseDuration(nextTLSID)
   
    #print(X,greenTime,greenDurTime,yellowDurTime,redDurTime)
    #print(lgc1)
    
    data = []
    
    ##先运行一步
    libsumo.simulationStep()
    stepNum += 1
    while requireStop == 0:
        libsumo.simulationStep()
        stepNum += 1
        
        ###交通灯设置
        ####https://sumo.dlr.de/daily/pydoc/traci._vehicle.html
        nextTLSID = "2"
        TLSPos = 0
        states = libsumo.trafficlight.getRedYellowGreenState(nextTLSID)
        timeT = libsumo.simulation.getCurrentTime()  # 当前时间
        nextSwitch = libsumo.trafficlight.getNextSwitch(nextTLSID)
        phaseName = libsumo.trafficlight.getPhaseName(nextTLSID)
        phase = libsumo.trafficlight.getPhase(nextTLSID)
        phaseDur = libsumo.trafficlight.getPhaseDuration(nextTLSID)
        curTime = timeT/1000
        phaseLeftTime = nextSwitch - curTime
        
      
         

        #车设置
        veh = "00003"
        vehPos = libsumo.vehicle.getPosition(veh)[0]
        speed = libsumo.vehicle.getSpeed(veh)
        vehMaxSpeed = libsumo.vehicle.getMaxSpeed(veh)

        laneID = libsumo.vehicle.getLaneID(veh)
        #meanSpeed = libsumo.lane.getLastStepMeanSpeed(laneID)
        laneMaxSpeed = libsumo.lane.getMaxSpeed(laneID)

        vehMaxSpeed = min(vehMaxSpeed, laneMaxSpeed)
        
        dist = TLSPos-vehPos
        arrivalTime1 = min(100,dist / (speed + 0.001))
        arrivalTime2 = min(100,dist / vehMaxSpeed)
        #arrivalTime3 = min(100,dist / (meanSpeed+0.001))

        #predictMeanSpeed1min = laneMaxSpeed  #1分钟前绿灯时,当前场景下目标车道的平均速度
        #arrivalTime4 = min(dist / predictMeanSpeed1min,100)
        
        ##

       
        
        if states == "r" and  speed<10/3.6:
            print("low speed")
            redLowSpeed = 1
            requireStop = 1
            
        if states == "G" and  speed<10/3.6:
            print("low speed")
            redLowSpeed= 1
            requireStop = 1

      
            
            
        if timeT/1000 > 30 or dist<=0:  # if time is over 30 second stoping the simulation
            requireStop = 1
        #name1 = ["当前红灯剩余时间","距离红灯的位置","当前速度","车道预计平均速度","预计到达时间1","预计到达时间2"]    
        laneAVGSpeed = 40/3.6
        data1 =[phaseLeftTime,dist,speed,40/3.6,dist/(speed+0.01),dist/laneAVGSpeed]
        
        #前面只有停止的车，不存在运动的车,没有的车位置为0
        #vehTmp = "00001"
        #vehPos1 = libsumo.vehicle.getPosition(vehTmp)[0]
        #vehTmp = "00002"
        #vehPos2 = libsumo.vehicle.getPosition(vehTmp)[0]
        #name2 = ["红灯前车辆1位置","车辆2位置","车辆3位置","车辆4位置","车辆5位置","车辆6位置","车辆7位置","车辆8位置","速度一直高于1m/s"]    
        #laneAVGSpeed = 60/3.6
        #data2 =[TLSPos-vehPos1,TLSPos-vehPos2,0,0,0,0,0,0]
        #print("#####tls state: %s,duration:%.3f,running time(ms):%d,passTLS:%d,dist:%d,speed:%.2f" %(states,phaseDur,timeT,passTLSNoDeAcc,dist,speed))
        #data.append(data1+data2)
        
        
        ##获得车灯前的车.注意为了简单.最大为8辆车
        name2 = ["红灯前车辆1位置","红灯前车辆1速度","车辆2位置","车辆2速度","车辆3位置","车辆3速度","车辆4位置","车辆4速度"]
        name3 = ["红灯前车辆5位置","红灯前车辆5速度","车辆6位置","车辆6速度","车辆7位置","车辆7速度","车辆8位置","车辆8速度"]
       
        data2 =[0] * 16;
        vehicles = libsumo.vehicle.getIDList();  # 当前在道路上的车辆
        vehListTmp = []
        for vehTmp in vehicles:
            tlsList = libsumo.vehicle.getNextTLS(vehTmp)
            if vehTmp == "00003" or len(tlsList) == 0
                continue
            tlsList = libsumo.vehicle.getNextTLS(vehTmp)
            tlsID_T, tlsIndex, distTmp, state = tlsList[0]
            speed = libsumo.vehicle.getSpeed(vehTmp)
            
            if tlsID_T == "2":
                vehListTmp.append([vehTmp,distTmp,speed]) 
                
        if len(vehListTmp)>0
            vehListTmp.sort(key=sortTakeSecond)
            for i in range(len(vehListTmp))
                data2[2*i]   = vehListTmp[i][1]
                data2[2*i+1] = vehListTmp[i][2]
                
        data.append(data1+data2)
                
             

                
       
    
    
    #####################################速度分段标记<10为0.10<= <20 为1 《=-20 《30 为2。30《 为3 
    speedFlag = -1
    veh = "00003"
    if redLowSpeed == 1:
        speedFlag = 0
        
    if redLowSpeed == 0:
        speed = libsumo.vehicle.getSpeed(veh)
        print("########################################speed:",speed)
        if speed>=10/3.6 and speed <20/3.6:
            speedFlag = 1
        if speed>=20/3.6 and speed <30/3.6:
            speedFlag = 2
        if speed>=30/3.6:
            speedFlag = 3
        print("############speedFlag:",speedFlag)
        print("############speedFlag:",speedFlag)
       
    
    ########################################
    time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    libsumo.close()
    print("###########################################################%s, sim %d close" %(time,i))
    name1 = ["当前红灯剩余时间","距离红灯的位置","当前速度","车道预计平均速度","预计到达时间1","预计到达时间2"]   
    name2 = ["红灯前车辆1位置","红灯前车辆1速度","车辆2位置","车辆2速度","车辆3位置","车辆3速度","车辆4位置","车辆4速度"]
    name3 = ["红灯前车辆5位置","红灯前车辆5速度","车辆6位置","车辆6速度","车辆7位置","车辆7速度","车辆8位置","车辆8速度"]
       
    nameTmp = name1+name2+name3
    writer.writerow(nameTmp)
    #print(len(data))
    for i in range(len(data)):
        rowTmp=data[i]
        #print(type(rowTmp))
        rowTmp.extend([speedFlag])
        #print(rowTmp)
        writer.writerow(rowTmp)
        dataAllSim.append(rowTmp)
    csv_file.close()
    
    
   
    
    if redLowSpeed == 0:
        counterNoLowSpeed= counterNoLowSpeed+1
        
print("finish numSim:%d,LowSpeed:%d" %(numSim,numSim-counterNoLowSpeed))

#########################################################################################################
name1 = ["当前红灯剩余时间","距离红灯的位置","当前速度","车道预计平均速度","预计到达时间1","预计到达时间2"]   
name2 = ["红灯前车辆1位置","红灯前车辆1速度","车辆2位置","车辆2速度","车辆3位置","车辆3速度","车辆4位置","车辆4速度"]
name3 = ["红灯前车辆5位置","红灯前车辆5速度","车辆6位置","车辆6速度","车辆7位置","车辆7速度","车辆8位置","车辆8速度"]
       
nameTmp = name1+name2+name3+"速度标志" 
name1 = ["redLightTime","distToRedLight","speed","laneAvgSpeed","arriveTime1","arriveTime2"]   
name2 = ["vehPos_1","vehSpeed_1","vehPos_2","vehSpeed_2","vehPos_3","vehSpeed_3","vehPos_4","vehSpeed_4"] 
name3 = ["vehPos_5","vehSpeed_5","vehPos_6","vehSpeed_6","vehPos_7","vehSpeed_7","vehPos_8","vehSpeed_8"] 

headers = name1+name2+name3+"speedFlag"
str1 = "./trainData/dataAllSim.csv"
with open(str1,'w')as f:
    f_csv = csv.writer(f)
    f_csv.writerow(headers)
    f_csv.writerows(dataAllSim)
#############################################################################################################    
!rm -rf ./trainData/redTLS*.csv
!zip -q  ./trainData/dataAllSim.zip ./trainData/dataAllSim.csv
#!rm -rf ./trainData/*.csv

In [ ]:
!rm -rf ./trainData/redTLS*.csv
!zip -q  ./trainData/dataAllSim.zip ./trainData/dataAllSim.csv
!rm -rf ./trainData/*.csv

In [ ]:
!rm -rf ./trainData/*.csv

In [ ]:
!git config --global user.email "lukeliuli@163.com"
!git config --global user.name "lukeliuli"
!git add .
!git commit -m  "tls OK, collectiong all and adding speedFlag"
!git push